In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET
import json
import xml

In [2]:
import pymongo

In [3]:
import xmltodict

In [4]:
folder = 'D:/Fichiers_dev/test_xml/'

In [5]:
tree = ET.parse(folder + 'extrait_declaration2.xml')

In [6]:
root = tree.getroot()

In [34]:
declarations_json = []

for declaration in root :
    # pour chaque declaration (1er niveau du parsage), on reconvertie en xml string decodé en utf-8
    declaration_parse = ET.tostring(declaration).decode('utf-8')
    # on convertie en json chaque declaration
    declaration_json = xmltodict.parse(declaration_parse)
    # on enregistre chaque json dans une liste
    declarations_json.append((declaration_json['declaration']))

In [35]:
def remplacer_espace_insecable(dictionnaire, cle, valeur):
    
    # si la valeur est une chaine, faire le remplacement
    dictionnaire[cle] = valeur.replace('\xa0', '')

In [36]:
def conversion_numerique(dictionnaire, cle, valeur):
    
    # conversion en numerique
    if (cle=='annee' or cle=='montant'): 
        dictionnaire[cle] = int(valeur)

In [37]:
def transformation_dictionnaire_recursive_fonction(dictionnaire, fonction):
    for cle, valeur in dictionnaire.items():

        # si la valeur est un dictionnaire, appel récursif
        if isinstance(valeur, dict):
                valeur_dict = valeur
                transformation_dictionnaire_recursive_fonction(dictionnaire=valeur_dict, fonction=fonction)

        # si la valeur est une liste, on boucle sur la liste, puis appel récusrsif
        if isinstance(valeur, list):
                for dictionnaire_in in valeur :
                    transformation_dictionnaire_recursive_fonction(dictionnaire=dictionnaire_in, fonction=fonction)

        if isinstance(valeur, str) :
            fonction(dictionnaire, cle, valeur)

        else : 
            dictionnaire[cle] = valeur

In [38]:
transformation_dictionnaire_recursive_fonction(dictionnaire=declarations_json[0], fonction=remplacer_espace_insecable)

In [40]:
# on applique les fonctions de transformation à toutes les déclarations

for declaration in declarations_json:
    # remplacer les espaces insécables
    transformation_dictionnaire_recursive_fonction(dictionnaire=declarations_json[0], fonction=remplacer_espace_insecable)
    # faire les conversions numériques
    transformation_dictionnaire_recursive_fonction(dictionnaire=declarations_json[0], fonction=conversion_numerique)

### Connection à MongoDB

In [41]:
client = pymongo.MongoClient('mongodb://localhost:27017')

In [42]:
db = client['declarations']

collection = db['declaration_extrait']

### Conversion en json et transfert dans MongoDB

In [45]:
for declaration in declarations_json:
    collection.insert_one(declaration)